<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Notion - Send LinkedIn invitation from database
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/LinkedIn/LinkedIn_Send_invitation_from_gsheet.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a>

**Tags:** #notion #invitation #automation #content #linkedin 

**Author:** [Florent Ravenel](https://www.linkedin.com/in/ACoAABCNSioBW3YZHc2lBHVG0E_TXYWitQkmwog/)

With this notebook, you will be able to send LinkedIn invitation from a Notion database.<br>
**NB:** Remember that LinkedIn limit invitations up to 100 per week (Becareful !)

## Input

### Import libraries

In [ ]:
import naas
from naas_drivers import notion, linkedin
import pandas as pd
import os
from datetime import datetime
import requests

### Setup Notion
- [Get your Notion integration token](https://docs.naas.ai/drivers/notion)
- Share integration with your database

In [ ]:
# Enter Token API
NOTION_TOKEN = "*****"
NOTION_TOKEN = naas.secret.get("NOTION_TOKEN_NAAS")

# Enter Database URL
DATABASE_URL = "https://www.notion.so/********"
DATABASE_URL = "https://www.notion.so/naas-official/6c37687f489a4384a359486e0fed54d9?v=185fb9b868024b238e06f3451a174e67"

# Column with Linkedin URL
col_lk_notion = 'Name'
col_lk_notion = 'LinkedIn'

### Setup LinkedIn
<a href='https://www.notion.so/LinkedIn-driver-Get-your-cookies-d20a8e7e508e42af8a5b52e33f3dba75'>How to get your cookies ?</a>

In [ ]:
# LinkedIn cookies
LI_AT = 'YOUR_COOKIE_LI_AT'
JSESSIONID = 'YOUR_COOKIE_JSESSIONID'
LI_AT = naas.secret.get("LI_AT")
JSESSIONID = naas.secret.get("JSESSIONID")

# LinkedIn limit invitations up to 100 per week (Becareful !)
LIMIT = 2

### Setup variables

In [ ]:
# CSV to manage and remove profile already in your contact
csv_contact = "LINKEDIN_EXISTING_CONTACT.csv"

# CSV to manage URL not valid
csv_not_valid = "LINKEDIN_NOT_VALID.csv"

# CSV to store invitations sent
csv_invitation = "LINKEDIN_INVITATIONS_SENT.csv"

### Schedule your notebook

In [ ]:
# Scheduler your invitation everyday at 8:00 AM
# naas.scheduler.add(cron="0 8 * * *")

# Uncomment the line below to delete your scheduler
# naas.scheduler.delete()

## Model

### Get Notion database

In [ ]:
db_notion = notion.connect(NOTION_TOKEN).database.get(DATABASE_URL)
df_notion = db_notion.df()
df_notion

### Get invitations sent

In [ ]:
df_invitation = linkedin.connect(LI_AT, JSESSIONID).invitation.get_sent()
df_invitation

### Get profile checked and already in your network

In [ ]:
def get_csv(output_path):
    df = pd.DataFrame()
    if os.path.exists(output_path):
        df = pd.read_csv(output_path)
    return df

In [ ]:
df_contacts = get_csv(csv_contact)
df_contacts

### Get invitations sent

In [ ]:
df_invitation_sent = get_csv(csv_invitation)
df_invitation_sent

### Get URL not valid

In [ ]:
df_not_valid = get_csv(csv_not_valid)
df_not_valid

### Get new invitation
- Clean Notion database to get valid URL
- Remove profile when already invited

In [ ]:
def get_new_invitations(df, df_invitation, df_contacts, df_not_valid, profile_id="PUBLIC_ID"):
    # Cleaning
    df = df[df[col_lk_notion].str.match(".+.com/in/.+")].reset_index(drop=True)
    df["PROFILE_ID"] = df.apply(lambda row: row[col_lk_notion].split("com/in/")[-1].split("/")[0], axis=1)
    
    # Remove profile already invited
    invitations_sent = df_invitation[profile_id].unique().tolist()
    contacts = []
    if len(df_contacts) > 0:
        contacts = df_contacts[profile_id].unique().tolist()
    not_valids = []
    if len(df_not_valid) > 0:
        not_valids = df_not_valid[col_lk_notion].unique().tolist()
    exclude = invitations_sent + contacts
    df = df[~df["PROFILE_ID"].isin(exclude)].reset_index(drop=True)
    return df

df_linkedin = get_new_invitations(df_notion, df_invitation, df_contacts, df_not_valid)
df_linkedin

### Send invitation

In [ ]:
def send_invitation(df, df_contacts=None, df_invitation=None, df_not_valid=None):
    # Setup variables
    df_network = pd.DataFrame()
    if df_contacts is None:
        df_contacts = pd.DataFrame()
    if df_invitation is None:
        df_invitation = pd.DataFrame()
    if df_not_valid is None:
        df_not_valid = pd.DataFrame()
        
    # Loop
    count = 1
    for index, row in df_linkedin.iterrows():
        profile = row[col_lk_notion]
        print(f"Bot started for :", profile)
        
        # Manage URL validity
        df_network = linkedin.connect(LI_AT, JSESSIONID).profile.get_network(profile)
        if type(df_network) == requests.exceptions.HTTPError:
            df_not_valid = pd.concat([df_not_valid, df])
            df_not_valid.to_csv(csv_not_valid, index=False)
            print("❌ URL not valid")
        # Manage Invitation
        else:
            if len(df_network) > 0:
                distance = df_network.loc[0, "DISTANCE"]
                if distance not in ["SELF", "DISTANCE_1"]:
                    linkedin.connect(LI_AT, JSESSIONID).invitation.send(recipient_url=profile)
                    print(count, "- 🙌 Invitation successfully sent")
                    df_invitation = pd.concat([df_invitation, df_network])
                    df_invitation["DATE_SENT"] = datetime.now().strftime("%Y-%m-%d")
                    df_invitation.to_csv(csv_invitation, index=False)
                    count += 1
                else:
                    df_contacts = pd.concat([df_contacts, df_network])
                    df_contacts.to_csv(csv_contact, index=False)
                    print(f"➡️ Already in my network, 💾 saved in CSV")
            
        # Manage LinkedIn limit
        if count > LIMIT:
            print("LinkedIn invitation limit reached", LIMIT)
            return df_invitation
        
df_invitation_sent = send_invitation(df_linkedin, df_contacts, df_invitation_sent)
df_invitation_sent

## Output

### Display result

In [ ]:
df_invitation_sent